In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from transformers import BertTokenizer

In [2]:
df=pd.read_excel('dataset_filledsupplier_currency_orderday.xlsx')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24621 entries, 0 to 24620
Data columns (total 32 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   청구서번호        24621 non-null  object 
 1   No.          24621 non-null  int64  
 2   Subject      24599 non-null  object 
 3   Machinery    24621 non-null  object 
 4   Assembly     24621 non-null  object 
 5   청구품목         24621 non-null  object 
 6   Unnamed: 6   0 non-null      float64
 7   Part No.1    24602 non-null  object 
 8   Part No.2    3592 non-null   object 
 9   청구량          24517 non-null  float64
 10  견적           24171 non-null  object 
 11  견적수량         24517 non-null  float64
 12  견적화폐         24621 non-null  object 
 13  견적단가         24621 non-null  float64
 14  발주번호         24621 non-null  object 
 15  발주처          24621 non-null  object 
 16  발주           24621 non-null  object 
 17  발주수량         24621 non-null  int64  
 18  발주금액         24621 non-null  float64
 19  D/T 

## 클리닝 - 마이너스값, 미입고기간 다지우고

In [4]:
missing_conditions = df[
    df['발주'].notnull() &  # 발주 일자는 비어있지 않음
    df['미입고 기간'].isnull() &  # 미입고 기간은 비어있음
    df['창고입고'].isnull() & # 창고 입고도 비어있음
    df['선박입고'].isnull()  # 선박 입고도 비어있음

]

print(f"발주 일자는 있지만 미입고 기간, 창고 입고, 선박 입고도 없는 경우: {len(missing_conditions)}개")
# 해당 조건의 행 삭제
df = df.drop(missing_conditions.index)

print(f"삭제된 행의 개수: {len(missing_conditions)}개")
print(f"남은 데이터프레임의 크기: {df.shape}")

발주 일자는 있지만 미입고 기간, 창고 입고, 선박 입고도 없는 경우: 1699개
삭제된 행의 개수: 1699개
남은 데이터프레임의 크기: (22922, 32)


In [5]:
#미입고기간으로 처리.
missing_both = df[df['창고입고'].isnull() & df['미입고 기간'].notnull()]

print(f"창고 입고일은 없고 미입고 기간은 명시되어 있어 미입고 기간으로 분류해야 할 경우 : {len(missing_both)}개")

창고 입고일은 없고 미입고 기간은 명시되어 있어 미입고 기간으로 분류해야 할 경우 : 1620개


In [53]:
# 1. '미입고 기간'이 없는 데이터만 필터링
df = df[df['미입고 기간'].isnull()].copy()
df = df[df['리드타임'] >= 0]
df = df.dropna(subset=['창고입고'])

print(df.shape)

(20934, 38)


# 리드타임 계산

> 1. 발주와 입고가 같은 경우 리드타임을 1로 설정
> 2. 발주 수량과 입고 수량이 같은 경우, 일반 계산으로 기본타입

In [54]:
df['발주']=pd.to_datetime(df['발주'], errors='coerce')
df['창고입고']=pd.to_datetime(df['창고입고'], errors='coerce')

df['리드타임']=(df['창고입고'] - df['발주']).dt.days
df['리드타임']=df['리드타임'].apply(lambda x:1 if x == 0 else x)

print(df[['발주', '창고입고', '리드타임']].tail(10))

              발주       창고입고  리드타임
24593 2022-03-31 2022-04-21    21
24594 2022-03-31 2022-04-21    21
24595 2022-03-31 2022-04-21    21
24596 2022-04-06 2022-07-03    88
24597 2022-04-06 2022-07-03    88
24598 2022-04-06 2022-07-03    88
24599 2022-04-06 2022-07-03    88
24600 2022-04-06 2022-07-03    88
24601 2022-04-06 2022-07-03    88
24602 2022-04-06 2022-07-03    88


In [42]:
print(df['리드타임'].notnull().sum(),df['리드타임'].isnull().sum())

20934 0


# 다중 출력 모델
### 리드타임 예측 (회귀) / 미입고 기간 예측 (분류) 
1. 텍스트 칼럼 결합 및 BERT 임베딩
2. ( 수치형 데이터(견적단가 및 발주량) Scaling )
3. 범주형 데이터(견적화폐) onehotEncoding
4. BERT 임베딩 유사도 => 모델의 입력, 2.3데이터 결합 => 리드타임 OR 미입고 기간 예측

### 전처리

In [80]:
import re

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\([^)]*\)', '', text)
    text = re.sub(r'[^\w\s\*/\-\+.,#&]', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'\b(사용금지|사)\b', '', text, flags=re.IGNORECASE)
    text = text.strip()
    return text

def clean_supplier_name(name):
    name = name.lower()
    name = re.sub(r'coporation|coropration|coproration|corporration', 'corporation', name)
    name = re.sub(r'\(사용금지\)', '', name)
    name = re.sub(r'u\.s\.a', '_usa', name)
    name = re.sub(r'\.', '', name)
    suffixes = r'(corporation|corp|company|co|incorporated|inc|limited|ltd|상사|공사|엔지니어링|주식회사|주|gmbh|pte ltd|llc)'
    name = re.sub(suffixes, '', name, flags=re.IGNORECASE)
    name = re.sub(r'[^\w\s-]', '', name)
    name = re.sub(r'\s+', ' ', name).strip()
    return name

In [81]:
# 텍스트 컬럼 리스트
text_columns = ['Machinery', 'Assembly']

for col in text_columns:
    df[col] = df[col].astype(str)
df['cleaned_machinery'] = df['Machinery'].apply(preprocess_text)
df['cleaned_assembly'] = df['Assembly'].apply(preprocess_text)

df['combined_text'] = (
    df['cleaned_machinery'].fillna('') + " " +
    df['cleaned_assembly'].fillna('') + " " 
)

In [60]:
from gensim.models import FastText
# 1. FastText 임베딩 학습
sentences = [text.split() for text in fdata['combined_text']]  # 텍스트 토큰화
fasttext_model = FastText(sentences, vector_size=100, window=3, min_count=1, epochs=10)

def get_fasttext_embeddings(text, model):
    words = text.split()
    embeddings = [model.wv[word] for word in words if word in model.wv]
    if embeddings:
        return np.mean(embeddings, axis=0)  # 단어 임베딩의 평균값을 사용
    else:
        return np.zeros(model.vector_size)

# Machinery와 Assembly 텍스트 임베딩 각각 계산
machinery_embeddings = np.array([get_fasttext_embeddings(text, fasttext_model) for text in df['cleaned_machinery']])
assembly_embeddings = np.array([get_fasttext_embeddings(text, fasttext_model) for text in df['cleaned_assembly']])


### 데이터 분할

In [74]:
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertModel
import torch
from torch.utils.data import DataLoader, TensorDataset 

# 2. 발주 날짜를 정수형으로 변환
df['발주_날짜_정수'] = (df['발주'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1D')

# 3. 견적화폐 원핫 인코딩
currency_ohe = OneHotEncoder(sparse_output=False)
currency_encoded = currency_ohe.fit_transform(df[['견적화폐']])

scaler = StandardScaler()
scaled_numeric_data = scaler.fit_transform(df[['발주_날짜_정수', '발주금액']].values)

# 2. 텍스트 임베딩 (FastText나 BERT)는 그대로 사용
X = np.concatenate([
    machinery_embeddings,  # FastText or BERT 임베딩 (스케일링 없이 사용)
    assembly_embeddings,   # FastText or BERT 임베딩 (스케일링 없이 사용)
    currency_encoded,      # 원핫 인코딩된 견적화폐
    scaled_numeric_data    # 스케일링된 발주 날짜와 발주 금액
], axis=1)

# 5. y 값: 도착 날짜 (창고입고)를 정수형으로 변환
y = (df['창고입고'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1D')


In [75]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [82]:
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1)).ravel()
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1)).ravel()

In [83]:
from xgboost import XGBRegressor

# 8. XGBoost 모델 학습
xgb_model = XGBRegressor()
xgb_model.fit(X_train_scaled, y_train_scaled)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [84]:
# 9. 예측
y_pred_scaled = xgb_model.predict(X_test_scaled)

# 10. 예측된 y 값을 원래 스케일로 역변환
y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).ravel()

# 11. 평가
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R^2 Score: {r2}")

Mean Squared Error (MSE): 704.0386944167905
Mean Absolute Error (MAE): 14.207189384553379
R^2 Score: 0.9947404861450195


In [87]:
mse = 700  
rmse = np.sqrt(mse)  

print(f"평균 오차 (RMSE): {rmse}일")

평균 오차 (RMSE): 26.457513110645905일


In [85]:
errors = np.abs(y_test - y_pred)
large_errors = df.iloc[np.argsort(-errors)][:10]  # 상위 10개의 오차가 큰 데이터
print(large_errors[['Machinery', 'Assembly', '발주', '창고입고', '리드타임']])

                         Machinery  \
635            NO.3 REF COMPRESSOR   
981   MAIN BOOM STBD VANG BLOCK(상)   
1740           NO.5 REF COMPRESSOR   
47               SPEED BOAT ENGINE   
3921         NO.3 GENERATOR ENGINE   
4112                   PURSE WINCH   
307                    PURSE WINCH   
2204            7M NET BOAT ENGINE   
3996            M/E REDUCTION GEAR   
3145         NO.1 GENERATOR ENGINE   

                                               Assembly         발주       창고입고  \
635   CYLINDER LINER AND CAPACITY CONTROL REPLACEMEN... 2019-01-17 2019-01-31   
981                                               BLOCK 2019-07-01 2019-07-03   
1740  CRANKSHAFT AND FRONT BEARING COVER REPLACEMENT GP 2019-05-08 2019-06-05   
47                               SPEED BOAT ENGINE ASSY 2019-04-08 2019-05-31   
3921                          318-5187 MTG GP-ENG FLEX  2019-06-04 2019-06-17   
4112                                    LEVER WIND ASSY 2019-08-08 2019-11-19   
307           